# lecture 12 - Named Entity Recognition (NER)
spaCy has an **'ner'** pipeline component that identifies token spans fitting a predetermined set of named entities. These are available as the `ents` property of a `Doc` object.

In [1]:
import spacy

In [2]:
nlp = spacy.load('en_core_web_sm')

In [9]:
def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_)))
    else:
        print('No named entities found.')

In [10]:
doc = nlp(u'Hi how are you?')

In [11]:
show_ents(doc)

No named entities found.


In [12]:
doc = nlp(u'May I go to Washington, DC next May to see the Washington Monument?')

In [14]:
show_ents(doc)

Washington, DC - GPE - Countries, cities, states
next May - DATE - Absolute or relative dates or periods
the Washington Monument - ORG - Companies, agencies, institutions, etc.


## Entity annotations
`Doc.ents` are token spans with their own set of annotations.
<table>
<tr><td>`ent.text`</td><td>The original entity text</td></tr>
<tr><td>`ent.label`</td><td>The entity type's hash value</td></tr>
<tr><td>`ent.label_`</td><td>The entity type's string description</td></tr>
<tr><td>`ent.start`</td><td>The token span's *start* index position in the Doc</td></tr>
<tr><td>`ent.end`</td><td>The token span's *stop* index position in the Doc</td></tr>
<tr><td>`ent.start_char`</td><td>The entity text's *start* index position in the Doc</td></tr>
<tr><td>`ent.end_char`</td><td>The entity text's *stop* index position in the Doc</td></tr>
</table>

In [15]:
doc = nlp(u'Can I please borrow 500 dollars from you to buy some Microsoft stock?')

In [16]:
show_ents(doc)

500 dollars - MONEY - Monetary values, including unit
Microsoft - ORG - Companies, agencies, institutions, etc.


## NER Tags
Tags are accessible through the `.label_` property of an entity.
<table>
<tr><th>TYPE</th><th>DESCRIPTION</th><th>EXAMPLE</th></tr>
<tr><td>`PERSON`</td><td>People, including fictional.</td><td>*Fred Flintstone*</td></tr>
<tr><td>`NORP`</td><td>Nationalities or religious or political groups.</td><td>*The Republican Party*</td></tr>
<tr><td>`FAC`</td><td>Buildings, airports, highways, bridges, etc.</td><td>*Logan International Airport, The Golden Gate*</td></tr>
<tr><td>`ORG`</td><td>Companies, agencies, institutions, etc.</td><td>*Microsoft, FBI, MIT*</td></tr>
<tr><td>`GPE`</td><td>Countries, cities, states.</td><td>*France, UAR, Chicago, Idaho*</td></tr>
<tr><td>`LOC`</td><td>Non-GPE locations, mountain ranges, bodies of water.</td><td>*Europe, Nile River, Midwest*</td></tr>
<tr><td>`PRODUCT`</td><td>Objects, vehicles, foods, etc. (Not services.)</td><td>*Formula 1*</td></tr>
<tr><td>`EVENT`</td><td>Named hurricanes, battles, wars, sports events, etc.</td><td>*Olympic Games*</td></tr>
<tr><td>`WORK_OF_ART`</td><td>Titles of books, songs, etc.</td><td>*The Mona Lisa*</td></tr>
<tr><td>`LAW`</td><td>Named documents made into laws.</td><td>*Roe v. Wade*</td></tr>
<tr><td>`LANGUAGE`</td><td>Any named language.</td><td>*English*</td></tr>
<tr><td>`DATE`</td><td>Absolute or relative dates or periods.</td><td>*20 July 1969*</td></tr>
<tr><td>`TIME`</td><td>Times smaller than a day.</td><td>*Four hours*</td></tr>
<tr><td>`PERCENT`</td><td>Percentage, including "%".</td><td>*Eighty percent*</td></tr>
<tr><td>`MONEY`</td><td>Monetary values, including unit.</td><td>*Twenty Cents*</td></tr>
<tr><td>`QUANTITY`</td><td>Measurements, as of weight or distance.</td><td>*Several kilometers, 55kg*</td></tr>
<tr><td>`ORDINAL`</td><td>"first", "second", etc.</td><td>*9th, Ninth*</td></tr>
<tr><td>`CARDINAL`</td><td>Numerals that do not fall under another type.</td><td>*2, Two, Fifty-two*</td></tr>
</table>

## Adding a Named Entity to a Span
Normally we would have spaCy build a library of named entities by training it on several samples of text.<br>In this case, we only want to add one value:

In [17]:
doc = nlp(u'Tesla to build a U.K. factory for $6 million')

In [18]:
show_ents(doc)

U.K. - GPE - Countries, cities, states
$6 million - MONEY - Monetary values, including unit


but Tesla is not recognised

In [19]:
from spacy.tokens import Span

In [20]:
ORG = doc.vocab.strings[u"ORG"]

In [21]:
ORG

383

In [22]:
new_ent = Span(doc,0,1,label=ORG)

In [23]:
doc.ents = list(doc.ents) + [new_ent]

In [25]:
show_ents(doc)

Tesla - ORG - Companies, agencies, institutions, etc.
U.K. - GPE - Countries, cities, states
$6 million - MONEY - Monetary values, including unit


so now Tesla has been recognised

## Adding Named Entities to All Matching Spans
What if we want to tag *all* occurrences of "Tesla"? In this section we show how to use the PhraseMatcher to identify a series of spans in the Doc:

In [29]:
doc = nlp(u'Our company created a brand new vacuum cleaner. '
          u'This new vacuum-cleaner is the best in show.')

In [30]:
show_ents(doc)

No named entities found.


In [31]:
from spacy.matcher import PhraseMatcher

In [32]:
mathcer = PhraseMatcher(nlp.vocab)

In [33]:
phrase_list = ['vacuum cleaner','vacuum-cleaner']

In [34]:
phrase_patterns = [nlp(text) for text in phrase_list]

In [35]:
mathcer.add('newproduct',None,*phrase_patterns)

In [36]:
found_matches = mathcer(doc)

In [37]:
found_matches

[(2689272359382549672, 6, 8), (2689272359382549672, 11, 14)]

In [38]:
from spacy.tokens import Span

In [39]:
PROD = doc.vocab.strings[u'PRODUCT']

In [41]:
new_ents = [Span(doc,match[1],match[2],label=PROD) for match in found_matches]

In [42]:
doc.ents = list(doc.ents) + new_ents

In [43]:
show_ents(doc)

vacuum cleaner - PRODUCT - Objects, vehicles, foods, etc. (not services)
vacuum-cleaner - PRODUCT - Objects, vehicles, foods, etc. (not services)


In [44]:
doc = nlp(u'Originally priced at $29.50, the sweater was marked down to five dollars.')

## Counting Entities
While spaCy may not have a built-in tool for counting entities, we can pass a conditional statement into a list comprehension:

In [45]:
[ent for ent in doc.ents if ent.label_ == 'MONEY']

[29.50, five dollars]

In [46]:
len([ent for ent in doc.ents if ent.label_ == 'MONEY'])

2